## Data Saving

In [ ]:
import os
import cv2
import json
import pickle
import face_recognition
from tkinter import filedialog, Tk
from fpdf import FPDF
from PIL import Image
import matplotlib.pyplot as plt

# ---------- Step 1: Collect student details ----------
print("Enter the following student details:")
name = input("Name: ").strip()
usn = input("USN: ").strip()
department = input("Department: ").strip()
current_semester = int(input("Current Semester (1-8): ").strip())
mobile = input("Mobile Number: ").strip()
blood_group = input("Blood Group: ").strip()

sgpas = []
percentages = []
cgpas = []

for sem in range(1, current_semester):
    while True:
        try:
            sgpa = float(input(f"Enter SGPA for Semester {sem} (e.g. 8.5): "))
            if 0 <= sgpa <= 10:
                sgpas.append(sgpa)
                break
            else:
                print("Please enter a valid SGPA between 0 and 10.")
                
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

    while True:
        try:
            perc = float(input(f"Enter percentage for Semester {sem} (e.g. 75.6): "))
            if 0 <= perc <= 100:
                percentages.append(perc)
                break
            else:
                print("Please enter a valid percentage between 0 and 100.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

    avg_sgpa = round(sum(sgpas) / len(sgpas), 2)
    cgpas.append(avg_sgpa)

cgpa = cgpas[-1] if cgpas else 0

# ---------- Step 2: Create folder for student ----------
base_path = "student_data"
folder_name = name.replace(" ", "_") + "_" + usn
student_folder = os.path.join(base_path, folder_name)
os.makedirs(student_folder, exist_ok=True)

# ---------- Step 3: Save student details ----------
details = {
    "Name": name,
    "USN": usn,
    "Department": department,
    "Current Semester": current_semester,
    "Mobile": mobile,
    "Blood Group": blood_group,
    "CGPA": cgpa,
    "SGPAs": {f"Sem {i+1}": sgpas[i] for i in range(len(sgpas))},
    "Percentages": {f"Sem {i+1}": percentages[i] for i in range(len(percentages))},
    "CGPAs": {f"Sem {i+1}": cgpas[i] for i in range(len(cgpas))}
}
with open(os.path.join(student_folder, "details.json"), "w") as f:
    json.dump(details, f, indent=4)

# ---------- Step 4: Capture 10 face images & full photo ----------
print(f"[INFO] Starting face capture for {name}. Press 'q' to quit early.")
video = cv2.VideoCapture(0)
count = 0
max_images = 10
face_encodings = []
full_photo_captured = False

while count < max_images:
    ret, frame = video.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)

    if face_locations:
        top, right, bottom, left = face_locations[0]
        face_image = frame[top:bottom, left:right]

        img_path = os.path.join(student_folder, f"face_{count+1}.jpg")
        cv2.imwrite(img_path, face_image)

        face_encoding = face_recognition.face_encodings(rgb_frame, [face_locations[0]])[0]
        face_encodings.append(face_encoding)

        if not full_photo_captured:
            full_photo = cv2.resize(frame, (140, 170))
            full_photo_path = os.path.join(student_folder, "full_photo.jpg")
            cv2.imwrite(full_photo_path, full_photo)
            full_photo_captured = True

        count += 1

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, f"{count}/{max_images}", (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    cv2.imshow("Face Registration", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

with open(os.path.join(student_folder, 'encodings.pkl'), 'wb') as f:
    pickle.dump(face_encodings, f)

print(f"[INFO] Saved {count} face images and encodings.")

# ---------- Step 5: Select mark card images ----------
print("[INFO] Select mark card images (one per semester)")
root = Tk()
root.withdraw()

image_paths = []
for sem in range(1, current_semester):
    print(f"Select image for Semester {sem}")
    file_path = filedialog.askopenfilename(
        title=f"Select image for Semester {sem}",
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )
    if file_path:
        ext = os.path.splitext(file_path)[1]
        dest_path = os.path.join(student_folder, f"sem{sem}_result{ext}")
        with open(file_path, 'rb') as src, open(dest_path, 'wb') as dst:
            dst.write(src.read())
        image_paths.append(dest_path)
    else:
        print(f"[WARNING] Semester {sem} image not selected. Skipping.")

# ---------- Step 6: Generate bar + line chart ----------
if percentages:
    semesters = [f"Sem {i}" for i in range(1, len(percentages) + 1)]
    plt.figure(figsize=(6.5, 4))
    bars = plt.bar(semesters, percentages, color='skyblue', label='Percentage')
    plt.plot(semesters, percentages, color='darkorange', marker='o', linewidth=2, label='Trend Line')
    plt.ylim(0, 100)
    plt.xlabel("Semester")
    plt.ylabel("Percentage (%)")
    plt.title("Semester-wise Percentage Overview")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend()

    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f"{yval:.1f}%", ha='center', va='bottom', fontsize=9)

    chart_path = os.path.join(student_folder, "percentage_chart.png")
    plt.tight_layout()
    plt.savefig(chart_path)
    plt.close()
else:
    chart_path = None

# ---------- Step 7: Generate PDF ----------
pdf = FPDF()
pdf.add_page()

# Logo
logo_path = r"C:\\Users\\TECHNO\\Desktop\\sem=6\\logo.png"
if os.path.exists(logo_path):
    pdf.image(logo_path, x=10, y=10, w=25)

# Heading
pdf.set_font("Times", "B", 22)
pdf.cell(200, 20, "PA COLLEGE OF ENGINEERING", ln=True, align="C")
pdf.set_font("Times", "B", 18)
pdf.cell(200, 10, "Academic Report", ln=True, align="C")
pdf.ln(8)

# Student details and photo
margin_left = 15
photo_width_mm = 38
photo_height_mm = 48
margin_top = pdf.get_y()
pdf.set_xy(margin_left, margin_top)
pdf.set_font("Times", "B", 13)
line_height = 8

initial_y = pdf.get_y()
for key, value in details.items():
    if key not in ["SGPAs", "Percentages", "CGPAs"]:
        pdf.cell(0, line_height, f"{key}: {value}", ln=True)
details_end_y = pdf.get_y()

# Photo
full_photo_path = os.path.join(student_folder, "full_photo.jpg")
if os.path.exists(full_photo_path):
    img = Image.open(full_photo_path)
    passport_img = img.resize((140, 170))
    passport_path = os.path.join(student_folder, "passport.jpg")
    passport_img.save(passport_path)

    photo_x = 210 - margin_left - photo_width_mm
    photo_y = initial_y
    pdf.rect(photo_x, photo_y, photo_width_mm, photo_height_mm)
    pdf.image(passport_path, photo_x + 1, photo_y + 1, photo_width_mm - 2, photo_height_mm - 2)

# Adjust spacing before table
pdf.set_y(max(details_end_y, photo_y + photo_height_mm) + 5)

# Academic table
pdf.set_font("Times", "B", 14)
pdf.cell(0, 10, "Academic Performance Summary", ln=True, align="C")
pdf.set_font("Times", "B", 12)
pdf.cell(50, 8, "Semester", 1, 0, "C")
pdf.cell(30, 8, "SGPA", 1, 0, "C")
pdf.cell(35, 8, "Percentage", 1, 0, "C")
pdf.cell(35, 8, "CGPA", 1, 1, "C")
pdf.set_font("Times", "", 12)

for i in range(len(sgpas)):
    pdf.cell(50, 8, f"Semester {i + 1}", 1, 0, "C")
    pdf.cell(30, 8, f"{sgpas[i]:.2f}", 1, 0, "C")
    pdf.cell(35, 8, f"{percentages[i]:.2f}%", 1, 0, "C")
    cgpa_str = "Nil" if i == 0 else f"{cgpas[i]:.2f}"
    pdf.cell(35, 8, cgpa_str, 1, 1, "C")

pdf.ln(5)

# Chart
if chart_path and os.path.exists(chart_path):
    chart_width_mm = 150  # Reduced from 180
    x_center = (210 - chart_width_mm) / 2
    y_chart = pdf.get_y()
    pdf.image(chart_path, x=x_center, y=y_chart, w=chart_width_mm)
    pdf.ln(60)

# Semester Results images
if image_paths:
    for i, img_path in enumerate(image_paths):
        try:
            image = Image.open(img_path)
            image_rgb = image.convert('RGB')
            temp_path = os.path.join(student_folder, f"temp_sem{i + 1}.jpg")
            image_rgb.save(temp_path)

            pdf.add_page()
            pdf.set_font("Times", "B", 14)
            pdf.cell(200, 10, f"Semester {i + 1} Marks Card", ln=True, align="C")
            pdf.image(temp_path, x=10, y=20, w=190)

            os.remove(temp_path)
        except Exception as e:
            print(f"[ERROR] Could not add image {img_path}: {e}")

# Output the PDF
output_pdf = os.path.join(student_folder, "Academic_Report.pdf")
pdf.output(output_pdf)
print(f"[INFO] Academic report generated at: {output_pdf}")


## With Prediction

In [6]:
import os
import cv2
import json
import pickle
import face_recognition
from tkinter import filedialog, Tk
from fpdf import FPDF
from PIL import Image
import matplotlib.pyplot as plt

# ---------- Step 1: Collect student details ----------
print("Enter the following student details:")
name = input("Name: ").strip()
usn = input("USN: ").strip()
department = input("Department: ").strip()
current_semester = int(input("Current Semester (1-8): ").strip())
mobile = input("Mobile Number: ").strip()
blood_group = input("Blood Group: ").strip()

sgpas = []
percentages = []
cgpas = []

for sem in range(1, current_semester):
    while True:
        try:
            sgpa = float(input(f"Enter SGPA for Semester {sem} (e.g. 8.5): "))
            if 0 <= sgpa <= 10:
                sgpas.append(sgpa)
                break
            else:
                print("Please enter a valid SGPA between 0 and 10.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

    while True:
        try:
            perc = float(input(f"Enter percentage for Semester {sem} (e.g. 75.6): "))
            if 0 <= perc <= 100:
                percentages.append(perc)
                break
            else:
                print("Please enter a valid percentage between 0 and 100.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

    avg_sgpa = round(sum(sgpas) / len(sgpas), 2)
    cgpas.append(avg_sgpa)

cgpa = cgpas[-1] if cgpas else 0

# ---------- Step 2: Create folder for student ----------
base_path = "student_data"
folder_name = name.replace(" ", "_") + "_" + usn
student_folder = os.path.join(base_path, folder_name)
os.makedirs(student_folder, exist_ok=True)

# ---------- Step 3: Save student details ----------
details = {
    "Name": name,
    "USN": usn,
    "Department": department,
    "Current Semester": current_semester,
    "Mobile": mobile,
    "Blood Group": blood_group,
    "CGPA": cgpa,
    "SGPAs": {f"Sem {i+1}": sgpas[i] for i in range(len(sgpas))},
    "Percentages": {f"Sem {i+1}": percentages[i] for i in range(len(percentages))},
    "CGPAs": {f"Sem {i+1}": cgpas[i] for i in range(len(cgpas))}
}

# ---------- Step 4: Predict next percentage ----------
try:
    model_path = os.path.join("model", "outcome_predictor.pkl")
    with open(model_path, "rb") as f:
        predictor = pickle.load(f)

    percs = percentages.copy()
    if len(percs) < 6:
        last_known = percs[-1] if percs else 0
        percs += [last_known] * (6 - len(percs))

    percs = percs[:6]
    predicted = predictor.predict([percs])[0]
    predicted = max(0, min(100, predicted))  # Clamp to 0–100
    details["Predicted Next Percentage"] = round(predicted, 2)
    print(f"[INFO] Predicted next semester percentage: {details['Predicted Next Percentage']}%")
except Exception as e:
    print(f"[WARNING] Could not predict percentage: {e}")
    details["Predicted Next Percentage"] = "N/A"

with open(os.path.join(student_folder, "details.json"), "w") as f:
    json.dump(details, f, indent=4)

# ---------- Step 5: Capture face images ----------
print(f"[INFO] Starting face capture for {name}. Press 'q' to quit early.")
video = cv2.VideoCapture(0)
count = 0
max_images = 10
face_encodings = []
full_photo_captured = False

while count < max_images:
    ret, frame = video.read()
    if not ret:
        continue

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)

    if face_locations:
        top, right, bottom, left = face_locations[0]
        face_image = frame[top:bottom, left:right]

        img_path = os.path.join(student_folder, f"face_{count+1}.jpg")
        cv2.imwrite(img_path, face_image)

        face_encoding = face_recognition.face_encodings(rgb_frame, [face_locations[0]])[0]
        face_encodings.append(face_encoding)

        if not full_photo_captured:
            full_photo = cv2.resize(frame, (140, 170))
            full_photo_path = os.path.join(student_folder, "full_photo.jpg")
            cv2.imwrite(full_photo_path, full_photo)
            full_photo_captured = True

        count += 1
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, f"{count}/{max_images}", (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    cv2.imshow("Face Registration", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

with open(os.path.join(student_folder, 'encodings.pkl'), 'wb') as f:
    pickle.dump(face_encodings, f)

# ---------- Step 6: Select mark card images ----------
print("[INFO] Select mark card images (one per semester)")
root = Tk()
root.withdraw()
image_paths = []
for sem in range(1, current_semester):
    print(f"Select image for Semester {sem}")
    file_path = filedialog.askopenfilename(
        title=f"Select image for Semester {sem}",
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )
    if file_path:
        ext = os.path.splitext(file_path)[1]
        dest_path = os.path.join(student_folder, f"sem{sem}_result{ext}")
        with open(file_path, 'rb') as src, open(dest_path, 'wb') as dst:
            dst.write(src.read())
        image_paths.append(dest_path)

# ---------- Step 7: Generate chart ----------
if percentages:
    semesters = [f"Sem {i}" for i in range(1, len(percentages) + 1)]
    plt.figure(figsize=(6.5, 4))
    bars = plt.bar(semesters, percentages, color='skyblue', label='Percentage')
    plt.plot(semesters, percentages, color='darkorange', marker='o', linewidth=2, label='Trend Line')
    plt.ylim(0, 100)
    plt.xlabel("Semester")
    plt.ylabel("Percentage (%)")
    plt.title("Semester-wise Percentage Overview")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend()
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f"{yval:.1f}%", ha='center', va='bottom', fontsize=9)
    chart_path = os.path.join(student_folder, "percentage_chart.png")
    plt.tight_layout()
    plt.savefig(chart_path)
    plt.close()
else:
    chart_path = None

# ---------- Step 8: Generate PDF ----------
pdf = FPDF()
pdf.add_page()
logo_path = r"C:\\Users\\TECHNO\\Desktop\\sem=6\\logo.png"
if os.path.exists(logo_path):
    pdf.image(logo_path, x=10, y=10, w=25)
pdf.set_font("Times", "B", 22)
pdf.cell(200, 20, "PA COLLEGE OF ENGINEERING", ln=True, align="C")
pdf.set_font("Times", "B", 18)
pdf.cell(200, 10, "Academic Report", ln=True, align="C")
pdf.ln(8)
margin_left = 15
photo_width_mm = 38
photo_height_mm = 48
margin_top = pdf.get_y()
pdf.set_xy(margin_left, margin_top)
pdf.set_font("Times", "B", 13)
line_height = 8
initial_y = pdf.get_y()
for key, value in details.items():
    if key not in ["SGPAs", "Percentages", "CGPAs"]:
        pdf.cell(0, line_height, f"{key}: {value}", ln=True)
details_end_y = pdf.get_y()
full_photo_path = os.path.join(student_folder, "full_photo.jpg")
if os.path.exists(full_photo_path):
    img = Image.open(full_photo_path)
    passport_img = img.resize((140, 170))
    passport_path = os.path.join(student_folder, "passport.jpg")
    passport_img.save(passport_path)
    photo_x = 210 - margin_left - photo_width_mm
    photo_y = initial_y
    pdf.rect(photo_x, photo_y, photo_width_mm, photo_height_mm)
    pdf.image(passport_path, photo_x + 1, photo_y + 1, photo_width_mm - 2, photo_height_mm - 2)
pdf.set_y(max(details_end_y, photo_y + photo_height_mm) + 5)
pdf.set_font("Times", "B", 14)
pdf.cell(0, 10, "Academic Performance Summary", ln=True, align="C")
pdf.set_font("Times", "B", 12)
pdf.cell(50, 8, "Semester", 1, 0, "C")
pdf.cell(30, 8, "SGPA", 1, 0, "C")
pdf.cell(35, 8, "Percentage", 1, 0, "C")
pdf.cell(35, 8, "CGPA", 1, 1, "C")
pdf.set_font("Times", "", 12)
for i in range(len(sgpas)):
    pdf.cell(50, 8, f"Semester {i + 1}", 1, 0, "C")
    pdf.cell(30, 8, f"{sgpas[i]:.2f}", 1, 0, "C")
    pdf.cell(35, 8, f"{percentages[i]:.2f}%", 1, 0, "C")
    cgpa_str = "Nil" if i == 0 else f"{cgpas[i]:.2f}"
    pdf.cell(35, 8, cgpa_str, 1, 1, "C")
pdf.ln(5)
if chart_path and os.path.exists(chart_path):
    chart_width_mm = 150
    x_center = (210 - chart_width_mm) / 2
    y_chart = pdf.get_y()
    pdf.image(chart_path, x=x_center, y=y_chart, w=chart_width_mm)
pdf.ln(60)
if image_paths:
    for i, img_path in enumerate(image_paths):
        try:
            image = Image.open(img_path)
            image_rgb = image.convert('RGB')
            temp_path = os.path.join(student_folder, f"temp_sem{i + 1}.jpg")
            image_rgb.save(temp_path)
            pdf.add_page()
            pdf.set_font("Times", "B", 14)
            pdf.cell(200, 10, f"Semester {i + 1} Marks Card", ln=True, align="C")
            pdf.image(temp_path, x=10, y=20, w=190)
            os.remove(temp_path)
        except Exception as e:
            print(f"[ERROR] Could not add image {img_path}: {e}")
output_pdf = os.path.join(student_folder, "Academic_Report.pdf")
pdf.output(output_pdf)
print(f"[INFO] Academic report generated at: {output_pdf}")


Enter the following student details:


Name:  MUMTHASIR K K
USN:  4PA22AI021
Department:  AIML
Current Semester (1-8):  6
Mobile Number:  9072002489
Blood Group:  B+ve
Enter SGPA for Semester 1 (e.g. 8.5):  7.3
Enter percentage for Semester 1 (e.g. 75.6):  70
Enter SGPA for Semester 2 (e.g. 8.5):  7.6
Enter percentage for Semester 2 (e.g. 75.6):  73
Enter SGPA for Semester 3 (e.g. 8.5):  6.9
Enter percentage for Semester 3 (e.g. 75.6):  66
Enter SGPA for Semester 4 (e.g. 8.5):  6.8
Enter percentage for Semester 4 (e.g. 75.6):  65
Enter SGPA for Semester 5 (e.g. 8.5):  7.7
Enter percentage for Semester 5 (e.g. 75.6):  75


[INFO] Predicted next semester percentage: 59.63%
[INFO] Starting face capture for MUMTHASIR K K. Press 'q' to quit early.
[INFO] Select mark card images (one per semester)
Select image for Semester 1
Select image for Semester 2
Select image for Semester 3
Select image for Semester 4
Select image for Semester 5
[INFO] Academic report generated at: student_data\MUMTHASIR_K_K_4PA22AI021\Academic_Report.pdf
